# Phase‑2: Inference & Evaluation of Fine‑Tuned Marketing KPI Model

This notebook demonstrates how to load the Phase‑1 LoRA fine‑tuned model and generate insights for unseen marketing KPI scenarios.

We include sample inputs and model outputs to illustrate real‑world usage.

## Step 0: Imports & Setup

In [1]:
!pip install -q transformers peft accelerate bitsandbytes torch

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.3 MB/s eta 0:00:00


2026-01-12 22:59:53.624556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768258794.041696      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768258794.169824      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768258795.114708      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768258795.114755      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768258795.114758      24 computation_placer.cc:177] computation placer alr

## Step 1: Load Model & Tokenizer

In [2]:
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
LORA_PATH = "/kaggle/input/marketing-kpi-lora-phase1/marketing_kpi_lora"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    use_fast=True
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

# Attach LoRA adapter
model = PeftModel.from_pretrained(model, LORA_PATH)

# Inference mode
model.eval()
print("✅ LoRA adapter loaded successfully")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

✅ LoRA adapter loaded successfully


## Step 2: Structured Prompt Template (Instruction Style)

In [3]:
def build_prompt(impressions, conversions, budget):
    return f"""### Instruction:
Analyze marketing campaign performance and provide actionable insights.

### Input:
Impressions: {impressions}, Conversions: {conversions}, Budget: ${budget}

### Response:
"""

## Step 3: Inference Function

In [4]:
@torch.no_grad()
def generate_insight(prompt, max_new_tokens=120):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## Step 4: Example Inference Inputs & Outputs
Below are some unseen KPI examples and the model’s generated responses.


In [5]:
# 3 examples in (impressions, conversations, budget)
examples = [
    (40000, 150, 2500),
    (80000, 200, 8000),
    (120000, 300, 10000),
]

# Inference loop over examples
for imp, conv, bud in examples:
    # Build prompt
    prompt = build_prompt(imp, conv, bud)
    print("Input:", prompt)

    # Model inference
    print("Output:", generate_insight(prompt))
    print("-" * 80)

Input: ### Instruction:
Analyze marketing campaign performance and provide actionable insights.

### Input:
Impressions: 40000, Conversions: 150, Budget: $2500

### Response:

Output: ### Instruction:
Analyze marketing campaign performance and provide actionable insights.

### Input:
Impressions: 40000, Conversions: 150, Budget: $2500

### Response:
Conversion rate is 0.38%. Conversion rate is low. Optimize creative and targeting.
--------------------------------------------------------------------------------
Input: ### Instruction:
Analyze marketing campaign performance and provide actionable insights.

### Input:
Impressions: 80000, Conversions: 200, Budget: $8000

### Response:

Output: ### Instruction:
Analyze marketing campaign performance and provide actionable insights.

### Input:
Impressions: 80000, Conversions: 200, Budget: $8000

### Response:
Conversion rate is 0.25%. Conversion rate is low. Optimize creative and targeting.
-------------------------------------------------

## Step 5: Simple Scoring

In [6]:
def simple_label(impressions, conversions):
    cr = conversions / impressions
    if cr < 0.005:
        return "Low"
    elif cr < 0.015:
        return "Moderate"
    else:
        return "Good"

for imp, conv, bud in examples:
    expected = simple_label(imp, conv)
    print(f"Expected category: {expected}")

Expected category: Low
Expected category: Low
Expected category: Low


## Evaluation Summary

The fine‑tuned model was able to generate relevant insights for unseen KPI scenarios.  
This demonstrates that the model learned patterns from the Phase‑1 synthetic dataset and produces reasonable recommendations for new inputs.

These examples can be shown to hiring managers or embedded in dashboards/APIs.